In [8]:
!pip install cloud-volume
!pip install caveclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.9/226.9 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of botocore to determine which version is compatible with other requirements. This could take a while.
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.0/238.0 kB 4.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 4.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from caveclient import CAVEclient, auth
from caveclient import chunkedgraph as cg

In [25]:
import numpy as np
import pandas as pd

In [167]:
url = "https://local.brain-wire-test.org"
global_url = "https://global.brain-wire-test.org/"
# datastack = "h01_c3_flat"
# dataset = "h01_full0_v2"
# datastack = "h01_test"
# dataset = "test0_parents_v0"
datastack = "fish1_full"
dataset = "fish1_full_v5"

## Create CAVEClient Client

In [168]:
client = CAVEclient(datastack_name=datastack, server_address=global_url)

## Create ChunkedGraph Client

In [169]:
cggraph = cg.ChunkedGraphClient(server_address=url, table_name=dataset, auth_client=auth.AuthClient(token=client.auth.token))

## Get Segmentation Volume Data 

In [170]:
# Generate a cloudvolume instance
cv = client.info.segmentation_cloudvolume()

In [171]:
cv

In [102]:
# load csv data
data = pd.read_csv("../data/proofread_test.csv", sep=',')

In [161]:
resolution = (8, 8, 30)

In [183]:
def get_latest_root_id(x, y, z):
    supervoxel_id = cv.download_point(pt=(x, y, z), 
                                      size=1, 
                                      agglomerate=False, 
                                      coord_resolution=resolution)
    supervoxel_id = np.int64(supervoxel_id[0, 0, 0, 0]) 
    print(supervoxel_id, type(supervoxel_id))
    root_id = cggraph.get_root_id(supervoxel_id)
    print(root_id, type(root_id))
    # latest_root_id = cggraph.get_latest_roots(root_id)
    # print(latest_root_id, type(latest_root_id))
    
    return root_id, supervoxel_id

In [ ]:
for index, row in data.iterrows():
    seg1_id, seg1_x, seg1_y, seg1_z, seg2_id, seg2_x, seg2_y, seg2_z = [*row]
    seg1_root_id, seg1_supervoxel_id = get_latest_root_id(seg1_x, seg1_y, seg1_z)
    seg2_root_id, seg2_supervoxel_id = get_latest_root_id(seg2_x, seg2_y, seg2_z)
    
    if seg1_root_id !== seg2_root_id:
        cggraph.do_merge(supervoxels=[seg1_supervoxel_id, seg2_supervoxel_id], 
                         coords=np.array([[seg1_x, seg1_y, seg1_z], [seg2_x, seg2_y, seg2_z]]),
                         resolution=resolution)

## Test run

In [186]:
seg1_x, seg1_y, seg1_z = 83582, 23488, 4343
seg2_x, seg2_y, seg2_z = 83571, 23456, 4359

In [187]:
seg1_root_id, seg1_supervoxel_id = get_latest_root_id(seg1_x, seg1_y, seg1_z)

Decompressing: 100%|█████████████████████████████| 1/1 [00:00<00:00, 922.64it/s]


83530793929934008 <class 'numpy.int64'>
864691128756038319 <class 'numpy.int64'>


In [188]:
seg2_root_id, seg2_supervoxel_id = get_latest_root_id(seg2_x, seg2_y, seg2_z)

Decompressing: 100%|████████████████████████████| 1/1 [00:00<00:00, 1483.66it/s]

83530793997041826 <class 'numpy.int64'>
864691128866780372 <class 'numpy.int64'>


In [193]:
np.array([[seg1_x, seg1_y, seg1_z], [seg2_x, seg2_y, seg2_z]])

array([[83582, 23488,  4343],
       [83571, 23456,  4359]])

In [194]:
if seg1_root_id != seg2_root_id:
    cggraph.do_merge(supervoxels=[seg1_supervoxel_id, seg2_supervoxel_id], 
                     coords=np.array([[seg1_x, seg1_y, seg1_z], [seg2_x, seg2_y, seg2_z]]),
                     resolution=resolution)

In [196]:
cv.download_point((83571, 23456, 4359), size=1, agglomerate=True, coord_resolution=(8,8,30))

Decompressing: 100%|█████████████████████████████| 1/1 [00:00<00:00, 888.81it/s]


VolumeCutout([[[[864691128914325444]]]], dtype=uint64)

https://ngl.brain-wire.org/#!%7B%22dimensions%22:%7B%22x%22:%5B8e-9%2C%22m%22%5D%2C%22y%22:%5B8e-9%2C%22m%22%5D%2C%22z%22:%5B3e-8%2C%22m%22%5D%7D%2C%22position%22:%5B83570.3515625%2C23466.435546875%2C4359.47021484375%5D%2C%22crossSectionScale%22:0.3191507890500759%2C%22projectionOrientation%22:%5B0.07601348310709%2C0.7280642986297607%2C0.13383030891418457%2C0.6680073738098145%5D%2C%22projectionScale%22:366.82187891949104%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://fish1-public/clahe_231218%22%2C%22tab%22:%22source%22%2C%22name%22:%22img%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22graphene://middleauth+https://local.brain-wire-test.org/segmentation/1.0/fish1_full_v5%22%2C%22tab%22:%22segments%22%2C%22segments%22:%5B%22864691128914325444%22%5D%2C%22segmentQuery%22:%22864691128802883064%22%2C%22name%22:%22fish1_full%22%7D%5D%2C%22showSlices%22:false%2C%22gpuMemoryLimit%22:2000000000%2C%22systemMemoryLimit%22:3000000000%2C%22selectedLayer%22:%7B%22visible%22:true%2C%22layer%22:%22fish1_full%22%7D%2C%22layout%22:%22xy-3d%22%2C%22selection%22:%7B%7D%7D